In [1]:
import warnings
warnings.filterwarnings('ignore')
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse


In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re


def parse_word(response):
    word = response.meta['word']

    # Extracting lexicons
    word['lexicon_0'] = response.xpath('//body/br[2]/following-sibling::text()').get()
    if word['lexicon_0'] and word['lexicon_0'].strip() == '':
        word['lexicon_0'] = response.xpath('//body/br[3]/following-sibling::text()').get()

    for i in range(1, 5):
        lex = response.xpath(f'//body/hr[{i}]/following-sibling::text()').get()
        if lex and lex.strip() != '':
            word[f'lexicon_{i}'] = lex.strip()
        else:
            word[f'lexicon_{i}'] = None

    # Check if the word is a verb (look for "verb" in lexicon_0, lexicon_1, lexicon_2, lexicon_3)
    verb_form = None
    for lexicon_key in ['lexicon_0', 'lexicon_1', 'lexicon_2', 'lexicon_3']:
        if word.get(lexicon_key) and 'verb' in word[lexicon_key]:
            verb_form = word[lexicon_key].strip().split(' ')[2]  # Extract the form after 'verb'
            break

    # If a verb form is detected, extract the corresponding translations from <span class="mg1">
    if verb_form:
        # Extracting only the second and subsequent <span class="mg1">
        translation_list = response.xpath(
            f'(//span[@class="bin" and text()="{verb_form}"]/following-sibling::span[@class="mg1"])[position()>1]/text()').getall()

        # Clean up translations: filter out numbers and empty strings
        cleaned_translations = []
        for t in translation_list:
            t = t.strip()
            if t and not re.match(r'^\d+$', t):  # Exclude pure numbers
                cleaned_translations.append(t)

        # Add up to 4 cleaned translations to the word dictionary
        for i, translation in enumerate(cleaned_translations[:4]):
            word[f'meaning_{i}'] = translation

        # Fill any remaining meaning columns with None
        for i in range(len(cleaned_translations), 4):
            word[f'meaning_{i}'] = None
    else:
        # Extract multiple word meanings for non-verbs (base logic unchanged)
        meanings = response.xpath('//span[@class="mgP"]/text()').getall()
        for i, meaning in enumerate(meanings):
            if i < 4:
                word[f'meaning_{i}'] = meaning.strip()

        # Fill any remaining meaning columns with None
        for i in range(len(meanings), 4):
            word[f'meaning_{i}'] = None

    yield word


class CalWordSpider(scrapy.Spider):
    name = 'cal'
    start_urls = [
        'https://cal.huc.edu/get_a_chapter.php?file=71009'
    ]

    def parse(self, response):
        for el in response.css('tr > td:nth-child(2) > a'):
            word = {
                'text': el.css('::text').get(),
                'url': el.xpath('@href').get()
            }
            yield scrapy.Request(
                url=f"https://cal.huc.edu/{word['url']}",
                meta={'word': word},
                callback=parse_word
            )


# Run the spider
process = CrawlerProcess(settings={
    'FEEDS': {
        'Masekhet_tan_checkk.json': {
            'format': 'json',
            'overwrite': True
        }
    }
})

process.crawl(CalWordSpider)
process.start()

2024-10-07 13:20:25 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2024-10-07 13:20:25 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 1.1.1t  7 Feb 2023), cryptography 39.0.1, Platform Windows-10-10.0.19045-SP0
2024-10-07 13:20:25 [scrapy.crawler] INFO: Overridden settings:
{}
2024-10-07 13:20:25 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2024-10-07 13:20:25 [scrapy.extensions.telnet] INFO: Telnet Password: e7328adfe7d27b09
2024-10-07 13:20:25 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2024-10-07 13:20:25 [scrapy.middleware] INFO: Enabled downloader mi

2024-10-07 13:20:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002210&word=5>
{'text': 'lh;', 'url': 'bablex.php?coord=7100901002210&word=5', 'lexicon_0': '\n\n\nl_ p02 ', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'to, for', 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002209&word=5> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:20:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002209&word=2> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:20:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002209&word=0> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:20:28 [scrapy.core.engine] DEBUG: Crawled (200) <GE

2024-10-07 13:20:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901010234&word=6>
{'text': 'r)wyyN"', 'url': 'bablex.php?coord=7100901010234&word=6', 'lexicon_0': 'there is no data for this word, it may be undeciphereable or simply not Aramaic', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': None, 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901010234&word=5>
{'text': '"dybwr', 'url': 'bablex.php?coord=7100901010234&word=5', 'lexicon_0': 'there is no data for this word, it may be undeciphereable or simply not Aramaic', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': None, 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901010234

2024-10-07 13:20:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901015242&word=3>
{'text': 'hy)', 'url': 'bablex.php?coord=7100901015242&word=3', 'lexicon_0': '\n\n\nhy) P01 ', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'she', 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901015245&word=7>
{'text': '$n)', 'url': 'bablex.php?coord=7100901015245&word=7', 'lexicon_0': '\n\n\n$ny verb G', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'to be different', 'meaning_1': 'to go away', 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901017229&word=2> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:20:29 [scrapy.core.scraper] DE

2024-10-07 13:20:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901019201&word=1> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:20:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901017238&word=8>
{'text': 'mw(d)', 'url': 'bablex.php?coord=7100901017238&word=8', 'lexicon_0': 'there is no data for this word, it may be undeciphereable or simply not Aramaic', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': None, 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:20:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901018207&word=5>
{'text': 'btlsr;', 'url': 'bablex.php?coord=7100901018207&word=5', 'lexicon_0': '\n\n\nb_ p02 ', 'lexicon_1': 'tlysr n01', 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'in, with', 'meaning_1': 'thirteen', 'meaning_2': None,

...

2024-10-07 13:23:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002239&word=2> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002245&word=0>
{'text': 'm)y', 'url': 'bablex.php?coord=7100901002245&word=0', 'lexicon_0': '\n\n\nm)y P01 ', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'what', 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:23:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002240&word=0>
{'text': 'zmnyhw;', 'url': 'bablex.php?coord=7100901002240&word=0', 'lexicon_0': '\n\n\nzymn) noun sg. emphatic', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'set time; instance', 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:23:06 [scrapy.cor

2024-10-07 13:23:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002238&word=1>
{'text': '"txyt', 'url': 'bablex.php?coord=7100901002238&word=1', 'lexicon_0': 'there is no data for this word, it may be undeciphereable or simply not Aramaic', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': None, 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:23:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002237&word=10>
{'text': ")m'", 'url': 'bablex.php?coord=7100901002237&word=10', 'lexicon_0': '\n\n\n)mr verb G', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'to say', 'meaning_1': 'to command', 'meaning_2': 'to think', 'meaning_3': 'idiomatic and special usages:'}
2024-10-07 13:23:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002237&word=11>
{'text': 'l

2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002215&word=3> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002215&word=2> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002215&word=1> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002215&word=0> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002216&word=2>
{'text': 'mtnyt)', 'url': 'bablex.php?coord=7100901002216&word=2', 'lexicon_0': '\n\n\nmtnyt) noun sg. emphatic', 'lexicon_1': Non

2024-10-07 13:23:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002208&word=5>
{'text': 'gmr', 'url': 'bablex.php?coord=7100901002208&word=5', 'lexicon_0': '\n\n\ngmr verb G', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'trans.', 'meaning_1': 'intrans.', 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002208&word=4> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002208&word=3> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://cal.huc.edu/bablex.php?coord=7100901002203&word=1> (referer: https://cal.huc.edu/get_a_chapter.php?file=71009)
2024-10-07 13:23:07 [scrapy.core.engine] DEBUG: Crawled (

2024-10-07 13:23:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002114&word=3>
{'text': 'dqt)ny;', 'url': 'bablex.php?coord=7100901002114&word=3', 'lexicon_0': '\n\n\nd_ c = d_ c --> dy c', 'lexicon_1': 'tny verb G', 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'to repeat', 'meaning_1': 'to narrate, recite', 'meaning_2': 'to sift flour [< Rabbinic Hebrew]', 'meaning_3': None}
2024-10-07 13:23:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://cal.huc.edu/bablex.php?coord=7100901002114&word=1>
{'text': 'hyky', 'url': 'bablex.php?coord=7100901002114&word=1', 'lexicon_0': '\n\n\nhyky a ', 'lexicon_1': None, 'lexicon_2': None, 'lexicon_3': None, 'lexicon_4': None, 'meaning_0': 'how', 'meaning_1': None, 'meaning_2': None, 'meaning_3': None}
2024-10-07 13:23:07 [scrapy.core.engine] INFO: Closing spider (finished)
2024-10-07 13:23:07 [scrapy.extensions.feedexport] INFO: Stored json feed (5990 items) in: Masekhet_ta